In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [2]:
path_1 = Path('/Users/oleseifert/Documents/Techlabs/SoSe-21-Team-10_Main/data/raw')
df = pd.read_csv(path_1.joinpath('2000-2002.csv'), index_col='date', parse_dates=True)

In [3]:
# drop the 'fips' column, because it is useless
df = df.drop(['fips'], axis=1)

# Interpolate the data

In [4]:
df_intpol = df.interpolate(method='time')
df_intpol.dropna(how='any', inplace=True)

# Create the test and the training set

In [12]:
X = df_intpol.drop(['score'], axis=1)
y = df_intpol.score

In [31]:
X_train = X.iloc[0:874,:]
y_train = y.iloc[0:874]
X_test = X.iloc[874:,:]
y_test = y.iloc[874:]

In [ ]:
# Create the random forest model

In [3]:
from sklearn.ensemble import RandomForestRegressor

In [34]:
model = RandomForestRegressor()

# Creating a base model

In [35]:
model.fit(X_train, y_train)

RandomForestRegressor()

In [36]:
model.score(X_test, y_test)

-1.0163301142674128

## Score seems like the model is performing quite bad. Before creating the manual split I was able to achive scores of 0.3

# Trying to tune the hyperparameters

In [37]:
print(model.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


# Trying it the 'traditional way'

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
model2 = RandomForestRegressor()
model2.fit(X_train1,y_train1)

RandomForestRegressor()

In [42]:
model2.score(X_test1,y_test1)

0.34971525932155134

In [43]:
model2_pred = model2.predict(X_test1)

In [45]:
plt.plot(df_intpol.index.iloc[874:], df_intpol.score, color='blue')
plt.plot(df_intpol.index.iloc[874:], model2_pred, color='red')
plt.show()

AttributeError: 'DatetimeIndex' object has no attribute 'iloc'

# Random Forest with bigger training set:

In [2]:
path_2 = Path('/Users/oleseifert/Documents/Techlabs/SoSe-21-Team-10_Main/data/raw')
df = pd.read_csv(path_2.joinpath('train_timeseries.csv'), index_col='date', parse_dates=True)

In [4]:
df_2 = df

In [7]:
df_2.interpolate(method='time', inplace=True)
df_2.dropna(how='any', inplace=True)

In [8]:
df_2.isnull().sum()

fips           0
PRECTOT        0
PS             0
QV2M           0
T2M            0
T2MDEW         0
T2MWET         0
T2M_MAX        0
T2M_MIN        0
T2M_RANGE      0
TS             0
WS10M          0
WS10M_MAX      0
WS10M_MIN      0
WS10M_RANGE    0
WS50M          0
WS50M_MAX      0
WS50M_MIN      0
WS50M_RANGE    0
score          0
dtype: int64

In [13]:
df_2.shape

(19300677, 20)

In [ ]:
plt.rcParams['agg.path.chunksize'] = 19300677
plt.plot(df_2.index, df_2.score)
plt.show()

In [ ]:
X_train_full, X_test_full, y_train_full, y_test_full = 